In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

https://www.passline.com/sitio/bebop-club -> 403
"https://entradasonline.com.ar/" -> 200 pero datos dinamicos

In [244]:
maura = "https://maura.mitiendanube.com/" #OK
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"}
soup_maura, status = create_conection(maura,headers)
status


200

In [2]:
from abc import ABC,abstractmethod
class Scrapper(ABC):
     def __init__(self,url):
          self.url = url

     def fetch_content(self,url=None):
          if not url:
               url =self.url
          try:
               response = requests.get(url,headers)
               if (response.status_code==200):
                    html_text = response.text
                    self.soup = BeautifulSoup(html_text,'html5lib') #library that makes it easy to scrape information from web pages
               else:
                    self.soup = None
               return self.soup,response.status_code
          except requests.RequestException as e:
               return f"Error haciendo el fetch de {url}: {e}",404
          
     

     def execute_scrapping(self,tag_container,class_container,tag_name) -> list[any]:
          pass

In [146]:
class Prenda:
     def __init__(self,nombre:str,precio):
          self.nombre = nombre
          self.precio = precio

     def __eq__(self,other):
          if not isinstance(other,type(self)):
               return NotImplemented
          return (self.nombre==other.nombre and self.precio==other.precio)        
     def __repr__(self) -> str:
          return f"Show(nombre={self.nombre},precio={self.precio})"
     
     def __hash__(self):
          return hash((self.nombre,self.precio))
     

from enum import Enum

class TipoPaginacion(Enum):
    PAGINADO_DINAMICO = "?mpage="
    PAGINADO_ESTATICO = "/page/"

class TiendaNubeScrapping(Scrapper):
    def __init__(self, url :str ,tag_sublinks,tag_container,a_class,subtag_name):
        super().__init__(url)
        self.tag_sublinks = tag_sublinks
        self.tag_container=tag_container
        self.a_class = a_class
        self.subtag_name = subtag_name

    def create_df(self,data_list):
          return pd.DataFrame([prenda.__dict__ for prenda in data_list])

    def execute_scrapping(self) -> list[Prenda]:
        prendas = set()     
        soup_maura_ = self.soup.find_all(self.tag_sublinks['tag'],class_=self.tag_sublinks['class'])
        soup_elements = soup_maura_[0].find_all('a',class_=self.a_class, href=True)
        for element in soup_elements:
          
                if element and element.has_attr('href') and element['href'] != '#' and len(element['href'])>15:
                        s2,status = self.fetch_content(element['href'])
                        if (status ==200):
                            print(element['href'])

                            soup = s2.find_all(self.tag_container['tag'],class_=self.tag_container['class'])
                            
                            for s in soup:
                                try:
                                    
                                    nombre = s.find_all(self.subtag_name,class_=re.compile('item-name'))[0].text.strip()
                                    precio = s.find_all('span',class_=re.compile('js-price-display item-price'))[0].text.strip()
                                    prenda = Prenda(nombre,precio)    
                                    
                                    prendas.add(prenda)    
                                except:
                                     print(f"No se pudo cargar en {element['href']}")
                        else:
                             print("No entró")        
        return list(prendas)





# Making scrapping examples

In [269]:
# Testing with kazuma

kazuma = TiendaNubeScrapping(url="https://kazuma.com.ar/")
_, status = kazuma.fetch_content()
if(status==200):
        list_kazuma =kazuma.execute_scrapping(
                tag_container='div',
                class_container='item-description',
                subtag_name='div')
        df_kazuma = kazuma.create_df(list_kazuma)
df_kazuma        


https://www.kazuma.com.ar/remeras/
https://kazuma.com.ar/remeras/oversize/
https://kazuma.com.ar/remeras/super-oversize/
https://kazuma.com.ar/remeras/regular-fit/


,nombre,precio
0,REMERA OVERSIZE PATTY,$16.900
1,REMERA OVERSIZE SCRIBBLE,$16.900
2,REMERA OVERSIZE SANDWICHES,$16.900
3,REMERA REGULAR MENTAL HEALTH,$13.000
4,REMERA SUPER OVERSIZE ROMEO,$14.890
5,REMERA OVERSIZE HEALTH FOOD,$16.900
6,REMERA OVERSIZE PLASTIC BAG,$16.900
7,REMERA REGULAR DOLPHIN,$12.500
8,REMERA UNIDENTIFIED,$11.550
9,REMERA BÁSICA REGULAR OSAKA II,$12.700


In [ ]:
#Tesitng with pda
pda = TiendaNubeScrapping(url="https://www.pda.com.ar")
pda.fetch_content()
list_pda = pda.execute_scrapping()
df_pda = pd.DataFrame([prenda.__dict__ for prenda in list_pda])
df_pda

# Create the instances and generate the complete dataframe of Tienda Nube
This will include implement the creation of the instances and then putting all together to make the scrapping

Concatenating the dataframes

In [19]:
def creacion_df_tienda(tienda):
        df_tienda = pd.DataFrame()
        _, status = tienda.fetch_content()
        if(status==200):
                list_tienda =tienda.execute_scrapping()
                df_tienda = tienda.create_df(list_tienda)
        else:
                print("No se pudo cargar la pagina")
        return df_tienda


In [13]:
def anexar_df_complete(tiendas,df_complete):
    for tienda in tiendas:
        df = creacion_df_tienda(tienda)
        df_complete = pd.concat([df_complete,df])
    return df_complete

In [139]:
pda = TiendaNubeScrapping(
    url="https://www.pda.com.ar",
    tag_sublinks ={"tag": "div","class": 'nav-primary'},
    tag_container={"tag": "div","class": 'item-description py-4 px-3'},
    a_class='nav-list-link',
    subtag_name='div')
kazuma = TiendaNubeScrapping(
    url="https://kazuma.com.ar",
    tag_sublinks ={"tag": "div","class": 'nav-primary'},
    tag_container={"tag": "div","class": 'item-description'},
    a_class='nav-list-link',
    subtag_name='div')
tiendas = [pda,kazuma]
df_tiendas_nube = pd.DataFrame()
df_tiendas_nube= anexar_df_complete(tiendas,df_tiendas_nube)
df_tiendas_nube

https://pda.com.ar/productos/
https://pda.com.ar/personalizado-con-tu-logo/
No se pudo cargar en https://pda.com.ar/personalizado-con-tu-logo/
https://www.pda.com.ar/personalizado-con-tu-logo/remeras/
https://www.pda.com.ar/personalizado-con-tu-logo/chombas/
No se pudo cargar en https://www.pda.com.ar/personalizado-con-tu-logo/chombas/
https://www.pda.com.ar/personalizado-con-tu-logo/buzos/
https://www.pda.com.ar/personalizado-con-tu-logo/camperas/
https://www.pda.com.ar/personalizado-con-tu-logo/articulos-polar/
https://www.pda.com.ar/personalizado-con-tu-logo/delantales/
https://www.pda.com.ar/personalizado-con-tu-logo/cofias/
https://www.pda.com.ar/personalizado-con-tu-logo/gorras/
https://www.pda.com.ar/personalizado-con-tu-logo/combos/
https://pda.com.ar/liso/
https://www.pda.com.ar/liso/zapatos/
https://www.pda.com.ar/liso/pantalones/
https://www.pda.com.ar/liso/buzos1/
https://www.pda.com.ar/liso/camperas1/
https://www.pda.com.ar/liso/articulos-polar1/
https://www.pda.com.ar/lis

,nombre,precio
0,Remera Verde,"$5.990,00"
1,Buzo Cuello Redondo Frisa Invisible Premium Negro,"$14.990,00"
2,Remera Blanca X 3,"$16.990,00"
3,Buzo Canguro Azul Frisa Invisible Premium,"$17.990,00"
4,Pantalon Cargo Jogger Gabardina,"$17.990,00"
...,...,...
98,CAMPERA SIRE APHOLO,$50.000
99,BUZO WAFFLE LIONEL,$29.890
100,JEAN REGULAR HENRY,$46.390
101,REMERA ONE DAY,$12.750


In [151]:
torm_facha = TiendaNubeScrapping(url="https://tormentadefacha.mitiendanube.com",
    tag_sublinks={"tag": "div","class": 'desktop-nav-container'},
    tag_container={"tag": "div","class": 'span3 item-container m-bottom-half'},
    a_class='desktop-nav-link',
    subtag_name='a')
tiendas = [torm_facha]
df_tiendas_nube= anexar_df_complete(tiendas,df_tiendas_nube)


https://tormentadefacha.mitiendanube.com/remeras/
https://tormentadefacha.mitiendanube.com/remeras/hombre/
https://tormentadefacha.mitiendanube.com/remeras/hombre/remeras2/
https://tormentadefacha.mitiendanube.com/remeras/hombre/volcom/
https://tormentadefacha.mitiendanube.com/remeras/hombre/remeras-sin-mangas-musculosas/
https://tormentadefacha.mitiendanube.com/remeras/hombre/vans1/
https://tormentadefacha.mitiendanube.com/remeras/hombre/rusty1/
https://tormentadefacha.mitiendanube.com/remeras/hombre/abercrombie/
https://tormentadefacha.mitiendanube.com/remeras/hombre/the-north-face/
https://tormentadefacha.mitiendanube.com/remeras/hombre/diesel/
https://tormentadefacha.mitiendanube.com/remeras/hombre/champion/
https://tormentadefacha.mitiendanube.com/remeras/hombre/salomon/
https://tormentadefacha.mitiendanube.com/remeras/hombre/rio-curl/
https://tormentadefacha.mitiendanube.com/remeras/hombre/santa-cruz-skate/
https://tormentadefacha.mitiendanube.com/remeras/mujer/
https://tormentad

In [ ]:
lemouton = TiendaNubeScrapping(url="https://lemoutonbebeshop.com.ar",
    tag_sublinks={"tag": "ul","class": "js-desktop-nav desktop-nav hidden-phone font-small"},
    tag_container={"tag": "div","class": 'span3 item-container m-bottom-half'},
    a_class='desktop-nav-link',
    subtag_name='a')
tiendas = [lemouton]
df_tiendas_nube= anexar_df_complete(tiendas,df_tiendas_nube)


In [ ]:
#Maura
maura = TiendaNubeScrapping(url="https://maura.mitiendanube.com",
    tag_sublinks={"tag": "div","class": 'row-fluid hidden-phone'},
    tag_container={"tag": "div","class": 'item-info-container m-top-half m-bottom-half'},
    a_class='desktop-nav-link',
    subtag_name='h6')
tiendas = [maura]
df_tiendas_nube= anexar_df_complete(tiendas,df_tiendas_nube)


In [152]:
df_tiendas_nube

,nombre,precio
0,Remera Verde,"$5.990,00"
1,Buzo Cuello Redondo Frisa Invisible Premium Negro,"$14.990,00"
2,Remera Blanca X 3,"$16.990,00"
3,Buzo Canguro Azul Frisa Invisible Premium,"$17.990,00"
4,Pantalon Cargo Jogger Gabardina,"$17.990,00"
...,...,...
51,Remera Rusty Classic,"$15.000,00"
52,Musculosa Vans Off the Wall,"$11.000,00"
53,Remera The North Face Training,"$15.000,00"
54,Remera Rusty Sunset Mountain mujer,"$15.000,00"


### Tests para nuevas paginas

In [ ]:
testing_url ="https://maura.mitiendanube.com"
tag_sublinks={"tag": "div","class": 'row-fluid hidden-phone'}
tag_container={"tag": "div","class": 'hidden-phone'}
subtag_name='h6'
a_class='desktop-nav-link'
#testing_url= "https://tormentadefacha.mitiendanube.com"
testing_instance = TiendaNubeScrapping(url=testing_url,
    tag_sublinks=tag_sublinks,
    tag_container=tag_container,
    subtag_name=subtag_name,
    a_class=a_class)
soup,_ = testing_instance.fetch_content()
soup


In [ ]:
soup1 = class_=soup.find_all('div','row-fluid hidden-phone')
soup_elements = soup1[0].find_all('a',class_='desktop-nav-link',href=True)
soup1

[<div class="head-top row-fluid hidden-phone" id="header">
 <div class="container">
 <div class="desktop-search p-relative span4 offset4">
 <div class="js-search-container searchbox">
 <form action="/search/" class="js-search-form" method="get">
 <input aria-label="¿Qué estás buscando?" autocomplete="off" class="js-search-input desktop-search-input" name="q" placeholder="¿Qué estás buscando?" type="search"/>
 <span class="desktop-search-icon">
 <svg baseProfile="tiny" class="svg-search-icon nav-svg" viewBox="598 -476.1 1792 1792" xmlns="http://www.w3.org/2000/svg"><path d="M1814 355.9c0-123.3-43.8-228.8-131.5-316.5-87.7-87.7-193.2-131.5-316.5-131.5s-228.8 43.8-316.5 131.5C961.8 127.1 918 232.6 918 355.9s43.8 228.8 131.5 316.5c87.7 87.7 193.2 131.5 316.5 131.5s228.8-43.8 316.5-131.5c87.7-87.7 131.5-193.2 131.5-316.5zm512 832c0 34.7-12.7 64.7-38 90s-55.3 38-90 38c-36 0-66-12.7-90-38l-343-342c-119.3 82.7-252.3 124-399 124-95.3 0-186.5-18.5-273.5-55.5s-162-87-225-150-113-138-150-225S662 45